In [1]:
import pandas as pd
import csv
import numpy as np

In [ ]:
PATH_TO_TRAIN_CSV = ""

In [2]:
df = pd.read_csv(PATH_TO_TRAIN_CSV = "")
print(df)

: 

: 

In [45]:
df.to_feather("train_df_raw.feather")

In [3]:
def parse_params(row_params: str):
    list_params = row_params.split("<br>")
    results = {}

    for line in list_params:
        if line == "":
            continue
        label_value = line.split("<=>")
        label = label_value[0]
        values = label_value[1].split("<->")
        if len(values) == 1 and values[0] == "":
            values = None
        results[label] = values
    
    return results
        

    

In [4]:
def find_all_params(df):
    all_params_labels = set()
    all_params_values = set()
    for row in df.itertuples(index=False): 
        params = parse_params(row[df.columns.get_loc('params')])
        labels = list(params.keys())
        values = [param for param in list(params.values()) if param != None]
        all_params_labels.update(labels)
        all_params_values.update(*values)
    
    return all_params_labels, all_params_values



In [29]:
def find_all_params_dict(df, labels):
    values = {}
    for label in labels:
        values[label] = set()
    
    for row in df.itertuples(index=False):
        params = parse_params(row[df.columns.get_loc('params')])
        for label in list(params.keys()):
            try:
                values[label].update(params[label])
                # print(*params[label])
            except:
                pass
    return values

In [28]:
labels_params, all_values = find_all_params(df)
with open('params_labels.txt','w') as f:
   f.write(str(labels_params))
with open('params_values.txt','w') as f:
   f.write(str(all_values))
values_params_dict = find_all_params_dict(df, labels_params)
with open('params_values_dict.txt','w') as f:
   for _ in values_params_dict.items():
      f.write(str(_))
      f.write("\n")

In [51]:
def create_df_advertisement(df):
    data = {}
    for label in labels_params:
        data[label] = []
         
    for idx, row in df.iterrows():
        results = parse_params(row["params"])
        for label in labels_params:
            try:
                data[label].append(results[label])
            except:
                data[label].append(None)
    return data

In [52]:
data = create_df_advertisement(df)
for label in list(data.keys()):
    if data[label] != None:
        df[label] = data[label]

In [54]:
df.iloc[1]

id                                                                513427
created_at_first                                     2019-02-18 12:17:51
category                                          Mieszkanie na sprzedaż
is_business                                                            1
district_id                                                          NaN
city_id                                                             3614
region_id                                                             15
description            <p>[b]Plewiska lokal 2-poziomowy w zabudowie s...
params                 price[currency]<=>PLN<br>m<=>95<br>rooms_num<=...
title                        Plewiska mieszkanie 2-poziomowe z ogródkiem
price                                                             500000
rent                                                                None
terrain_area                                                        None
building_type                                      

In [ ]:
df.to_feather("train_df_with_params.feather")
df.to_csv("train_df_with_params.csv")